ALLEEN Basic Preprocessing

In [1]:
import pandas as pd
import re
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import RandomOverSampler
import torch.nn.functional as F
import os 

In [2]:
# ✅ 1. Load & Preprocess Data
script_dir = os.getcwd() # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")


# 1. Dataset inladen
file_path = os.path.join(data_folder, "Test_data.xlsx")
df = pd.read_excel(file_path)

In [3]:
print(f"Number of rows Before filtering: {len(df)}")

print(df.head())

Number of rows Before filtering: 6064
                                             context  \
0  Hondenfokkers  -  Socialisatie pupsEén van de ...   
1  Hondenfokkers  -  Socialisatie pupsEén van de ...   
2  Hondenfokkers  -  Socialisatie pupsEén van de ...   
3  Hondenfokkers  -  Socialisatie pupsEén van de ...   
4  Hondenfokkers  -  Socialisatie pupsEén van de ...   

                                            question  statistical  \
0  Het gaat niet over illegale  kwekers)? b)Zo ja...            0   
1  2.a)Bestaat er een duidelijke omschrijving van...            0   
2  Zo niet, op basis van welke concrete  gegevens...            1   
3  3.a)Vinden er controles plaats door de dienst ...            0   
4  c)Hoeveel controles werden verricht in de peri...            1   

           theme    file_name  
0  Dierenwelzijn  1735311.txt  
1  Dierenwelzijn  1735311.txt  
2  Dierenwelzijn  1735311.txt  
3  Dierenwelzijn  1735311.txt  
4  Dierenwelzijn  1735311.txt  


In [4]:
# Drop unnecessary columns
if "TXT_file_name" in df.columns:
    df = df.drop(columns=["TXT_file_name"])

# Handle missing values
df = df.dropna(subset=["question"])
df["context"].fillna("", inplace=True)

# Clean text
def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Replace newlines with spaces
    text = re.sub(r'\b[a-z]\)\s*', ' ', text)  # Remove patterns like 'a)', 'b)', etc., with optional spaces
    text = re.sub(r'\b\d+\.\b', '', text)  # Remove patterns like '1.', '2.', etc.
    text = re.sub(r'\b\d+\)\b', '', text)  # Remove patterns like '1)', '2)', etc.
    text = re.sub(r'\b[i]+[.)]\b', '', text, flags=re.IGNORECASE)  # Remove patterns like 'i.', 'ii.', 'i)', etc.
    text = re.sub(r'\b\d+[.)]\s*', '', text) # Remove numeric list markers like 1., 2. or 1) 2)
    text = re.sub(r'\b[ivxlcdm]+\s*[.)]\s*', '', text, flags=re.IGNORECASE)# Remove roman numerals like i. ii. iii. or i) ii) iii)
    text = re.sub(r'•', '', text)  # Remove bullet symbol
    text = re.sub(r'\[\d+\]', '', text)  # Remove patterns like '[1]', '[2]', etc.
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and trim
    return text

# df["clean_text"] = (df["context"] + " " + df["question"]).apply(clean_text)
df["clean_text"] = (df["question"]).apply(clean_text) 

# ✅ Now: drop rare themes using original theme names
theme_counts = df["theme"].value_counts()
valid_themes = theme_counts[theme_counts >= 2].index
df = df[df["theme"].isin(valid_themes)]


C:\Users\jefva\AppData\Local\Temp\ipykernel_34360\1056078359.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["context"].fillna("", inplace=True)


In [5]:



# Clean text
def clean_text(text):
    text = re.sub(r'\n', ' ', text)  # Replace newlines with spaces
    text = re.sub(r'\b[a-z]\)\s*', ' ', text)  # Remove patterns like 'a)', 'b)', etc., with optional spaces
    text = re.sub(r'\b\d+\.\b', '', text)  # Remove patterns like '1.', '2.', etc.
    text = re.sub(r'\b\d+\)\b', '', text)  # Remove patterns like '1)', '2)', etc.
    text = re.sub(r'\b[i]+[.)]\b', '', text, flags=re.IGNORECASE)  # Remove patterns like 'i.', 'ii.', 'i)', etc.
    text = re.sub(r'\b\d+[.)]\s*', '', text) # Remove numeric list markers like 1., 2. or 1) 2)
    text = re.sub(r'\b[ivxlcdm]+\s*[.)]\s*', '', text, flags=re.IGNORECASE)# Remove roman numerals like i. ii. iii. or i) ii) iii)
    text = re.sub(r'•', '', text)  # Remove bullet symbol
    text = re.sub(r'\[\d+\]', '', text)  # Remove patterns like '[1]', '[2]', etc.
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and trim
    return text

# df["clean_text"] = (df["context"] + " " + df["question"]).apply(clean_text)
df["clean_text"] = (df["question"]).apply(clean_text) 



In [6]:
#amount of rows 
print(f"Number of rows after filtering: {len(df)}")

Number of rows after filtering: 6064


In [7]:
output_path = os.path.join(data_folder, "Test_data.xlsx")
df.to_excel(output_path, index=False)

#Totale data set gewoon gepreprocesseerd

removes rows where the same clean_text appears with multiple different theme values + Themes worden samengezet naar minder themas

In [8]:
# Group by 'clean_text' and count unique themes
duplicates_with_diff_themes = df.groupby("clean_text")["theme"].nunique().reset_index()

# Filter rows where the number of unique themes is greater than 1
duplicates_with_diff_themes = duplicates_with_diff_themes[duplicates_with_diff_themes["theme"] > 1]

# Merge back with the original dataframe to get all rows with these 'clean_text'
filtered_df = df[df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]
# Exclude rows with these 'clean_text' from the original dataframe
df = df[~df["clean_text"].isin(duplicates_with_diff_themes["clean_text"])]


In [9]:
print(f"Number of rows after filtering: {len(df)}")

Number of rows after filtering: 5888


In [10]:
#geeft aantal themas dat er in totaal zijn
print(f"Number of unique themes: {len(df['theme'].unique())}")


Number of unique themes: 34


In [11]:
# Thema-cluster mapping
theme_merge_map = {
    # Bestuur en Beleid
    "Lokale overheden en Binnenlands bestuur": "Bestuur en Beleid",
    "Vlaamse administratie": "Bestuur en Beleid",
    "Staatshervorming en Verhoudingen binnen de Belgische federale staat": "Bestuur en Beleid",
    "Vlaamse Regering": "Bestuur en Beleid",
    "Vlaams Parlement": "Bestuur en Beleid",

    # Mobiliteit en Infrastructuur
    "Mobiliteit en Verkeer": "Mobiliteit en Infrastructuur",
    "Openbare werken": "Mobiliteit en Infrastructuur",
    "Ruimtelijke ordening": "Mobiliteit en Infrastructuur",

    # Economie en Arbeid
    "Werk": "Economie en Arbeid",
    "Economie": "Economie en Arbeid",
    "Sociale economie": "Economie en Arbeid",
    "Internationaal ondernemen": "Economie en Arbeid",

    # Welzijn en Gezondheid
    "Welzijn en Gezin": "Welzijn en Gezondheid",
    "Gezondheid": "Welzijn en Gezondheid",
    "Armoedebeleid": "Welzijn en Gezondheid",

    # Cultuur en Communicatie
    "Cultuur": "Cultuur en Communicatie",
    "Media": "Cultuur en Communicatie",
    "Taalgebruik": "Cultuur en Communicatie",

    # Onderwijs en Samenleving
    "Onderwijs en Vorming": "Onderwijs en Samenleving",
    "Gelijke kansen": "Onderwijs en Samenleving",
    "Jeugdbeleid": "Onderwijs en Samenleving",
    "Integratie en Inburgering": "Onderwijs en Samenleving",

    # Milieu en Landbouw
    "Natuur en Milieu": "Milieu en Landbouw",
    "Landbouw": "Milieu en Landbouw",
    "Dierenwelzijn": "Milieu en Landbouw",

    # Internationaal Beleid
    "Buitenlands beleid": "Internationaal Beleid",
    "Europese instellingen": "Internationaal Beleid",
    "Ontwikkelingssamenwerking": "Internationaal Beleid",
    "Oekraïnecrisis": "Internationaal Beleid",

    # Overige (apart laten tenzij weinig samples)
    "Financiën": "Financiën",
    "Begroting": "Begroting",
    "Wetenschap en Innovatie": "Wetenschap en Innovatie",
    "Toerisme": "Toerisme",
    "Justitie en Handhaving": "Justitie en Handhaving",
    "Brussel en de Vlaamse Rand": "Brussel en de Vlaamse Rand",
    "Sport": "Sport",
    "Onroerend erfgoed": "Onroerend erfgoed",
    "Energie": "Energie",
    "Wonen": "Wonen",
}
# Nieuwe kolom aanmaken met samengevoegde thema's
df["theme"] = df["theme"].map(theme_merge_map).fillna("Onbekend")

In [12]:
#geeft aantal themas dat er in totaal zijn
print(f"Number of unique themes: {len(df['theme'].unique())}")
# Print the unique themes
print("Unique themes after mapping:")
print(df["theme"].unique())


Number of unique themes: 18
Unique themes after mapping:
['Milieu en Landbouw' 'Bestuur en Beleid' 'Onderwijs en Samenleving'
 'Mobiliteit en Infrastructuur' 'Toerisme' 'Energie'
 'Brussel en de Vlaamse Rand' 'Welzijn en Gezondheid' 'Financiën'
 'Economie en Arbeid' 'Wetenschap en Innovatie' 'Cultuur en Communicatie'
 'Begroting' 'Sport' 'Wonen' 'Justitie en Handhaving'
 'Internationaal Beleid' 'Onroerend erfgoed']


In [13]:
output_path = os.path.join(data_folder, "Test_data_NoDupsLessThemes.xlsx")
df.to_excel(output_path, index=False)

#Removes rows where the same clean_text appears with multiple different theme values + Themes worden samengezet naar minder themas

Remove rows with sentences less than 9 words (and zin: Kan de minister een overzicht geven)

In [14]:
# Remove rows where the 'question' column has 5 or fewer words
df = df[df['clean_text'].apply(lambda x: len(str(x).split()) > 9)]

# Remove rows where the 'question' column contains the specific phrase (with flexible matching)
df = df[~df['clean_text'].str.contains(r'\bKan de minister een overzicht geven\b', flags=re.IGNORECASE, na=False)]

# Remove rows where 'clean_text' contains "https:"
df = df[~df['clean_text'].str.contains(r'https:', flags=re.IGNORECASE, na=False)]

In [15]:
print(f"Number of rows after filtering: {len(df)}")

Number of rows after filtering: 4151


In [16]:
output_path = os.path.join(data_folder, "Test_data_NoDupsLessThemesENnoWords9.xlsx")
df.to_excel(output_path, index=False)

#Remove rows with sentences less than 9 words (and zin: Kan de minister een overzicht geven)

KLEINERE DATASET MAKEN van de 3 vorige (voor sneller testing)

In [24]:
# ✅ 1. Load & Preprocess Data
script_dir = os.getcwd()  # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_cleaned.xlsx")
df = pd.read_excel(file_path)

# Neem willekeurig 30.000 rijen
sample_df = df.sample(n=30000, random_state=42)

# Bepaal pad naar de bestaande 'Klein' map
originele_map = os.path.dirname(file_path)  # << hier was de fout
klein_map = os.path.join(originele_map, "Klein")

# Pad naar output-bestand
output_pad = os.path.join(klein_map, "Grote_data_cleaned_30000.xlsx")

# Sla het sample op
sample_df.to_excel(output_pad, index=False)


In [25]:
# ✅ 1. Load & Preprocess Data
script_dir = os.getcwd()  # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_NoDupsLessThemes.xlsx")
df = pd.read_excel(file_path)

# Neem willekeurig 30.000 rijen
sample_df = df.sample(n=30000, random_state=42)

# Bepaal pad naar de bestaande 'Klein' map
originele_map = os.path.dirname(file_path)  # << hier was de fout
klein_map = os.path.join(originele_map, "Klein")

# Pad naar output-bestand
output_pad = os.path.join(klein_map, "Grote_data_NoDupsLessThemes_30000.xlsx")

# Sla het sample op
sample_df.to_excel(output_pad, index=False)

In [27]:

# ✅ 1. Load & Preprocess Data
script_dir = os.getcwd()  # Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, "Grote_data_NoDupsLessThemesENnoWords9.xlsx")

df = pd.read_excel(file_path)

# Neem willekeurig 30.000 rijen
sample_df = df.sample(n=30000, random_state=42)

# Bepaal pad naar de bestaande 'Klein' map
originele_map = os.path.dirname(file_path)  # << hier was de fout
klein_map = os.path.join(originele_map, "Klein")

# Pad naar output-bestand
output_pad = os.path.join(klein_map, "Grote_data_NoDupsLessThemesENnoWords9_30000.xlsx")

# Sla het sample op
sample_df.to_excel(output_pad, index=False)